In [1]:
import math
from typing import Tuple, List

import pandas as pd
import mip

from src import paths


# Task 0 - name / prize / weight

#### Data

In [2]:
task0_data = pd.read_csv(paths.Datasets.PRIZE_WEIGHT_TSV, sep='\t')
task0_data

,name,prize,weight
0,Laptop,10,0.50
1,Flashcard,5,0.01
2,Cryptokey,1,0.01
3,Lamp,1,5.00
4,Battery,1,0.05
5,Charger,0,0.01


#### Solve function:

In [3]:
def solve_task0(data, max_weight, verbose=False):
    model = mip.Model()
    model.verbose=verbose

    variables = [
        model.add_var(name=item['name'], var_type=mip.BINARY)
        for _id, item in task0_data.iterrows()
    ]

    weights = data['weight']
    prizes = data['prize']

    weight_constr = mip.xsum(weight * var for var, weight in zip(variables, weights)) <= max_weight
    model.add_constr(weight_constr, 'weight')

    model.objective = mip.maximize(
        mip.xsum(prize * var for var, prize in zip(variables, prizes))
    )
    
    model.optimize()
    return model


In [4]:
def print_report(data, model):
    variables = model.vars
    data = data.copy()
    data['coef'] = [var.x for var in variables]
    
    used_weight = round(sum(data["coef"] * data["weight"]), 6)
    weight_constr = model.constr_by_name("weight")
    print('\n\n======= REPORT =======')
    print(f'Solution status: {model.status}')
    print(f'Total prize: {model.objective.x}')
    print(f'Used weight: {used_weight} / {weight_constr.rhs}')
    print(data)

### Experiments
Let's start with a single experiment with max_weight = $0.1$:

In [5]:
max_weight = 0.1
model = solve_task0(task0_data, max_weight=max_weight, verbose=True)
print_report(task0_data, model)

Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 24 2021 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 1 (0) rows, 5 (-1) columns and 5 (-1) elements
Clp1000I sum of infeasibilities 0 - average 0, 5 fixed columns
Coin0506I Presolve 0 (-1) rows, 0 (-5) columns and 0 (-5) elements
Clp0000I Optimal - objective value -0
Clp0000I Optimal - objective value -0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0000I Optimal - objective value 7.6
Clp0000I Optimal - objective value 7.6
Clp0000I Optimal - objective value 7.6
Coin0511I After Postsolve, objective 7.6, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 7.6 - 0 iterations time 0.002, Presolve 0.00

Starting MIP optimization


======= REPORT =======
Solution status: OptimizationStatus.OPTIMAL
Total prize: 7.0
Used weight: 0.07 / 0.1
        name  prize  weight  coef
0     Laptop     10    0.50   0.0
1  Flash

Seem like the solution is correct. Let's try some other weights:

In [6]:
max_weights = [0.01, 0.02, 0.4, 0.52, 6.0]
for max_weight in max_weights:
    model = solve_task0(task0_data, max_weight=max_weight, verbose=False)
    print_report(task0_data, model)



======= REPORT =======
Solution status: OptimizationStatus.OPTIMAL
Total prize: 5.0
Used weight: 0.01 / 0.01
        name  prize  weight  coef
0     Laptop     10    0.50   0.0
1  Flashcard      5    0.01   1.0
2  Cryptokey      1    0.01   0.0
3       Lamp      1    5.00   0.0
4    Battery      1    0.05   0.0
5    Charger      0    0.01   0.0


======= REPORT =======
Solution status: OptimizationStatus.OPTIMAL
Total prize: 6.0
Used weight: 0.02 / 0.02
        name  prize  weight  coef
0     Laptop     10    0.50   0.0
1  Flashcard      5    0.01   1.0
2  Cryptokey      1    0.01   1.0
3       Lamp      1    5.00   0.0
4    Battery      1    0.05   0.0
5    Charger      0    0.01   0.0


======= REPORT =======
Solution status: OptimizationStatus.OPTIMAL
Total prize: 7.0
Used weight: 0.07 / 0.4
        name  prize  weight  coef
0     Laptop     10    0.50   0.0
1  Flashcard      5    0.01   1.0
2  Cryptokey      1    0.01   1.0
3       Lamp      1    5.00   0.0
4    Battery      1   

# Task 1 - name / prize / weight + dependencies

#### Read dependencies data

In [7]:
Dependency = Tuple[List[str], str]

with open(paths.Datasets.DEPENDENCIES_TXT, 'r') as dependecies_fs:
    deps : List[Dependency] = [] 
    for line in dependecies_fs:
        tokens = line.rstrip('\n').split('\t')
        deps_target = (tokens[:-1], tokens[-1])
        deps.append(deps_target)

print(deps)

[(['Cryptokey'], 'Flashcard'), (['Flashcard'], 'Laptop'), (['Battery', 'Charger'], 'Laptop')]


Let's formulate how to convert the dependencient into the form of linear constraints.
Assume there's a dependency: $(A_1 | A_2 | \dots | A_n) \rightarrow B$, which means we can take the item B only provided at least on of the items $A_i$ have been taken. If we rewrite it into boolean expression, it will be a little different: $B \Rightarrow (A_1 \vee A_2 \vee \dots \vee A_n)$. It may be counterintuitive, but we can check it using the following intuition: if $B$ wasn't taken, there are no restrictions on $A_i$. Otherwise, there's a constraint that some $A_i$ must be non-zero.

Using the same logic, we can rewrite it into the CNF:
$B \Rightarrow (A_1 \vee A_2 \vee \dots \vee A_n) = (\neg B \vee A_1 \vee A_2 \vee \dots \vee A_n)$. We can check this equation for three variables using the CNF normaliser:

In [8]:
from sympy.logic.boolalg import to_cnf
from sympy.abc import A, B, C, D

to_cnf(D >> (A | B | C))

A | B | C | ~D

As a linear programming inequality it will look like:
$((1 - B) + A_1 + A_2 + \dots + A_n) \geqslant 1$. Let's code it:

#### Solve function:

In [9]:
def solve_task1(data, deps, max_weight, verbose=False):
    model = mip.Model()
    model.verbose=verbose

    # <<<< The same as in task 0 -- start >>>>
    variables = [
        model.add_var(name=item['name'], var_type=mip.BINARY)
        for _id, item in task0_data.iterrows()
    ]

    weights = data['weight']
    prizes = data['prize']

    weight_constr = mip.xsum(weight * var for var, weight in zip(variables, weights)) <= max_weight
    model.add_constr(weight_constr, 'weight')

    model.objective = mip.maximize(
        mip.xsum(prize * var for var, prize in zip(variables, prizes))
    )
    # <<<< The same as in task 0 -- end >>>>
    
    # Dependencies processing
    for dep_vars, target_var in deps:
        a_sum_term = mip.xsum([model.var_by_name(var) for var in dep_vars])
        b_term = 1.0 - model.var_by_name(target_var)
        
        dep_constr = (a_sum_term + b_term) >= 1.0
        model.add_constr(dep_constr)
    
    model.optimize()
    return model


In [10]:
max_weight = 0.5
model = solve_task1(task0_data, deps, max_weight=max_weight, verbose=True)
print_report(task0_data, model)

Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 4 (0) rows, 6 (0) columns and 13 (0) elements
Clp1000I sum of infeasibilities 0 - average 0, 3 fixed columns
Coin0506I Presolve 2 (-2) rows, 3 (-3) columns and 6 (-7) elements
Clp0006I 0  Obj 15.599917 Dual inf 241.42135 (2)
Clp0029I End of values pass after 3 iterations
Clp0000I Optimal - objective value 15.6
Clp0000I Optimal - objective value 15.6
Coin0511I After Postsolve, objective 15.6, infeasibilities - dual 0 (0), primal 0 (0)
Clp0006I 0  Obj 15.6
Clp0000I Optimal - objective value 15.6
Clp0000I Optimal - objective value 15.6
Clp0000I Optimal - objective value 15.6
Clp0032I Optimal objective 15.6 - 0 iterations time 0.

It seems like it is working - we were unable to take the laptop due to constraints. Let's try some other values:

In [11]:
max_weights = [0.01, 0.02, 0.51, 0.52, 0.53, 1.0]
for max_weight in max_weights:
    model = solve_task1(task0_data, deps, max_weight=max_weight, verbose=False)
    print_report(task0_data, model)



======= REPORT =======
Solution status: OptimizationStatus.OPTIMAL
Total prize: 1.0
Used weight: 0.01 / 0.01
        name  prize  weight  coef
0     Laptop     10    0.50   0.0
1  Flashcard      5    0.01   0.0
2  Cryptokey      1    0.01   1.0
3       Lamp      1    5.00   0.0
4    Battery      1    0.05   0.0
5    Charger      0    0.01   0.0


======= REPORT =======
Solution status: OptimizationStatus.OPTIMAL
Total prize: 6.0
Used weight: 0.02 / 0.02
        name  prize  weight  coef
0     Laptop     10    0.50   0.0
1  Flashcard      5    0.01   1.0
2  Cryptokey      1    0.01   1.0
3       Lamp      1    5.00   0.0
4    Battery      1    0.05   0.0
5    Charger      0    0.01   0.0


======= REPORT =======
Solution status: OptimizationStatus.OPTIMAL
Total prize: 7.0
Used weight: 0.07 / 0.51
        name  prize  weight  coef
0     Laptop     10    0.50   0.0
1  Flashcard      5    0.01   1.0
2  Cryptokey      1    0.01   1.0
3       Lamp      1    5.00   0.0
4    Battery      1  

# Task 2 - name / prize / weight + oversized items

In [12]:
task2_items = pd.read_csv(paths.Datasets.PRIZE_WEIGHT_VOLUME_TSV, sep='\t')
task2_items

,name,prize,weight,volume
0,Laptop,10,0.50,0.20
1,Flashcard,5,0.01,0.02
2,Cryptokey,1,0.01,0.02
3,Lamp,1,5.00,1.00
4,Battery,1,0.05,0.06
5,Charger,0,0.01,0.06


In [13]:
oversized_data = pd.read_csv(paths.Datasets.OVERSIZED_TSV, sep='\t')
oversized_data

,name,constraint
0,Laptop,0.05
1,Lamp,0.01


In [14]:
task2_data = pd.merge(task2_items, oversized_data, how='outer', left_on='name', right_on='name')
task2_data

,name,prize,weight,volume,constraint
0,Laptop,10,0.50,0.20,0.05
1,Flashcard,5,0.01,0.02,NaN
2,Cryptokey,1,0.01,0.02,NaN
3,Lamp,1,5.00,1.00,0.01
4,Battery,1,0.05,0.06,NaN
5,Charger,0,0.01,0.06,NaN


While the first constraint is straightforward, the second one is a bit more complicated. For each oversized item let's consider two cases. If we do not take it, there are no constraints. Otherwice, there's an upper bound on the volume for each taken item. Let $B$ is a single constrained item, $A_i$ are all the other items, $V_i$ is the volume and $C_b$ is the constraint boundary:

$ \neg B \vee ((A_1 \cdot V_1 \leqslant C_b) \wedge (A_2 \cdot V_2 \leqslant C_b) \wedge \dots \wedge (A_n \cdot V_n \leqslant C_b)) $ ($C_b$ here is the upper border value for $B$)

Let's use $ M $-value technique. Assuming $M$ is some large value (equivalent to infinity): 

$ (A_1 \cdot V_1 \leqslant (1 - B) \cdot M + C_b) \wedge (A_2 \cdot V_2 \leqslant (1 - B) \cdot M + C_b) \wedge \dots \wedge (A_n \cdot V_n \leqslant (1 - B) \cdot M + C_b) $ 

We can check it: if $B$ is equal to zero, we get $A_n \cdot V_n \leqslant M + C_b$, which is always true

Othewise, we get $A_n \cdot V_n \leqslant C_b$ which is true only we do not take the item or its volume is smaller than the constraint.

Finally, we can refactor this to:

$ \forall i = 1..n \;\; (A_i \cdot V_i \leqslant (1 - B) \cdot M + C_b) $

Assuming there are multiple constrained items $ B_j $:

$ \forall j = 1..m, \; \forall i = 1..n, \; i \neq j \;\; (A_i \cdot V_i \leqslant (1 - B) \cdot M + C_{B_j}) $


#### Solve function

In [15]:
def solve_task2(data, max_weight, max_volume=None, verbose=False):
    model = mip.Model()
    model.verbose=verbose

    # <<<< The same as in task 0 -- start >>>>
    variables = [
        model.add_var(name=item['name'], var_type=mip.BINARY)
        for _id, item in task0_data.iterrows()
    ]

    weights = data['weight']
    prizes = data['prize']

    weight_constr = mip.xsum(weight * var for var, weight in zip(variables, weights)) <= max_weight
    model.add_constr(weight_constr, 'weight')

    model.objective = mip.maximize(
        mip.xsum(prize * var for var, prize in zip(variables, prizes))
    )
    # <<<< The same as in task 0 -- end >>>>
    
    # Max volume constraint
    volumes = data['volume']
    
    if max_volume is not None:
        volume_constr = mip.xsum(volume * var for var, volume in zip(variables, volumes)) <= max_volume
        model.add_constr(volume_constr, 'volume')
    
    # Oversized items constraints
    M = 1e5
    volume_constraints = data['constraint']
    for constr_var, constraint in zip(variables, volume_constraints):
        if not math.isnan(constraint):
            for var, volume in zip(variables, volumes):
                if str(var) != str(constr_var):
                    if verbose:
                        print(f'Adding constraint: {var} * {volume} <= (1 - {constr_var}) * M + {constraint}')
                    constr = var * volume <= (1 - constr_var) * M + constraint 
                    model.add_constr(constr, f'oversized{constr_var}_{var}')
    
    model.optimize()
    return model


In [16]:
max_weight = 0.6
model = solve_task2(task2_data, max_weight=max_weight, verbose=True)
print_report(task2_data, model)

Adding constraint: Flashcard * 0.02 <= (1 - Laptop) * M + 0.05
Adding constraint: Cryptokey * 0.02 <= (1 - Laptop) * M + 0.05
Adding constraint: Lamp * 1.0 <= (1 - Laptop) * M + 0.05
Adding constraint: Battery * 0.06 <= (1 - Laptop) * M + 0.05
Adding constraint: Charger * 0.06 <= (1 - Laptop) * M + 0.05
Adding constraint: Laptop * 0.2 <= (1 - Lamp) * M + 0.01
Adding constraint: Flashcard * 0.02 <= (1 - Lamp) * M + 0.01
Adding constraint: Cryptokey * 0.02 <= (1 - Lamp) * M + 0.01
Adding constraint: Battery * 0.06 <= (1 - Lamp) * M + 0.01
Adding constraint: Charger * 0.06 <= (1 - Lamp) * M + 0.01
Cgl0003I 1 fixed, 0 tightened bounds, 3 strengthened rows, 0 substitutions
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Starting solution of the Linear programming relaxation probl

It seems to be working: we didn't add the charger even though there's enough volume and weight available.

In [17]:
max_weights = [0.02, 0.50, 0.52, 5.0]
for max_weight in max_weights:
    model = solve_task2(task2_data, max_weight=max_weight, verbose=False)
    print_report(task2_data, model)



======= REPORT =======
Solution status: OptimizationStatus.OPTIMAL
Total prize: 6.0
Used weight: 0.02 / 0.02
        name  prize  weight  volume  constraint  coef
0     Laptop     10    0.50    0.20        0.05   0.0
1  Flashcard      5    0.01    0.02         NaN   1.0
2  Cryptokey      1    0.01    0.02         NaN   1.0
3       Lamp      1    5.00    1.00        0.01   0.0
4    Battery      1    0.05    0.06         NaN   0.0
5    Charger      0    0.01    0.06         NaN   0.0


======= REPORT =======
Solution status: OptimizationStatus.OPTIMAL
Total prize: 10.0
Used weight: 0.5 / 0.5
        name  prize  weight  volume  constraint  coef
0     Laptop     10    0.50    0.20        0.05   1.0
1  Flashcard      5    0.01    0.02         NaN   0.0
2  Cryptokey      1    0.01    0.02         NaN   0.0
3       Lamp      1    5.00    1.00        0.01   0.0
4    Battery      1    0.05    0.06         NaN   0.0
5    Charger      0    0.01    0.06         NaN   0.0


======= REPORT ======